In [ ]:
import os
import sys
if sys.path[-1] != "../":
    sys.path.append("../")
    os.chdir("../")

import numpy as np
from IPython.display import display
from random import sample
from transformers import AutoModel, AutoTokenizer, AutoModelForConditionalGeneration

import torch
from utils.util import *
from utils.index import *
from utils.data import *

from hydra import initialize, compose

config = Config()
with initialize(version_base=None, config_path="../data/config/"):
    overrides = [
        "base=NQ320k",
        # "base=MS300k",
    ]
    hydra_config = compose(config_name="_example", overrides=overrides)
    config._from_hydra(hydra_config)

loaders = prepare_data(config)

loader_text = loaders["text"]
loader_query = loaders["query"]
text_dataset = loader_text.dataset
query_dataset = loader_query.dataset

In [ ]:
# load terms
code_type = "words_comma_plus_stem"
code_tokenizer = "t5"
# for NQ320k
code_length = 26
# for MS300k
# code_length = 34

tokenizer = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, code_tokenizer))

text_codes = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()

In [ ]:
indices = range(10)
text_code = text_codes[indices]
text_code[text_code == -1] = 0
display(tokenizer.batch_decode(text_code))
display(tokenizer.batch_decode(np.array(text_dataset[indices]["text"]["input_ids"])[:, :100]))